In [2]:
import os
import json
import re
import jieba

from opencc import OpenCC
from os import listdir
from gensim.models import word2vec

def sc_to_tc(string): #簡體中文轉繁體中文
    cc = OpenCC('s2t') 
    return cc.convert(string)

def remain_language(string): #留下文字
    result = re.sub('\W+', ' ', string).replace("_",  '')
    return result

def transform_and_write_data(file_path): #把wiki_zh濃縮成一個.txt
    files_layer_1 = listdir(file_path)
    #print(files_layer_1)
    
    with open( "./wiki_zh_seq.txt" , 'w', encoding='utf-8') as new_file: 
        for file_layer_1 in files_layer_1:
            if file_layer_1=='.DS_Store':
                continue
            else:
                files_layer_2 = listdir(file_path+"/"+file_layer_1)
                #print(files_layer_2)
                
                for file_layer_2 in files_layer_2:
                    with open( file_path+"/"+file_layer_1+"/"+file_layer_2 , 'r', encoding='utf-8') as file: #開wiki_zh的每個txt檔
                        for times, data in enumerate(file, 1):
                            #print('data num:', times)
                            data = json.loads(data)
                            data = data['text']
                            data = remain_language(data)
                            data = sc_to_tc(data) #轉繁體中文
                            data = jieba.lcut(data) 
                            data = [word for word in data if word != ' ']
                            data = ' '.join(data)+'\n'

                            new_file.write(data)


/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [26]:
def training(train_data_path): #訓練並儲存模型
    
    seed = 666 #亂數種子
    sg = 0 #用CBOW(0)或是Skip-gram(1)
    window_size = 10 #周圍詞彙要看多少範圍
    vector_size = 100 #轉成向量的維度
    min_count = 1 #詞頻少於 min_count 之詞彙不會參與訓練
    workers = 8 #訓練的並行數量
    epochs = 5 #訓練的迭代次數
    batch_words = 10000 #每次給予多少詞彙量訓練

    train_data = word2vec.LineSentence(train_data_path)
    
    model = word2vec.Word2Vec(
        train_data,
        min_count=min_count,
        vector_size=vector_size,
        workers=workers,
        epochs=epochs,
        window=window_size,
        sg=sg,
        seed=seed,
        batch_words=batch_words
    )

    model.save('word2vec.model') #儲存模型


In [32]:
file_path="./wiki_zh"
train_data_path="./wiki_zh_seq.txt"

transform_and_write_data(file_path) #檔案整理,並輸出

training(train_data_path) #訓練模型

model = word2vec.Word2Vec.load('word2vec.model')

In [50]:
while True:
    print("請輸入一個詞")
    ITME = input()
    
    if ITME=="quit":
        break
    else:
        print()
        print(ITME,"的 top 20 相關詞")
        try:
            for item in model.wv.most_similar(ITME, topn=20):
                print(item)
            print()
        except:
            print("不存在此詞語, 請重新輸入")
            print()

請輸入一個詞
女友

女友 的 top 20 相關詞
('男友', 0.9716050028800964)
('女朋友', 0.9047173857688904)
('男朋友', 0.9024293422698975)
('前女友', 0.8973594307899475)
('前男友', 0.8904036283493042)
('未婚夫', 0.8621507883071899)
('未婚妻', 0.8408583998680115)
('太太', 0.8189927935600281)
('老公', 0.8066743016242981)
('前妻', 0.7974343299865723)
('老婆', 0.794506847858429)
('分手', 0.7941772937774658)
('前夫', 0.7890509366989136)
('相戀', 0.7803738117218018)
('室友', 0.7784057855606079)
('閨蜜', 0.7688823342323303)
('熱戀', 0.7677774429321289)
('表哥', 0.7641410231590271)
('暗戀', 0.7624026536941528)
('外遇', 0.7583946585655212)

請輸入一個詞
老公

老公 的 top 20 相關詞
('老婆', 0.9025986194610596)
('女友', 0.8066743016242981)
('奶奶', 0.8038423657417297)
('太太', 0.7993292808532715)
('小強', 0.795569121837616)
('男友', 0.7941868901252747)
('閨蜜', 0.7906380891799927)
('媽媽', 0.7836593985557556)
('外遇', 0.7714070677757263)
('富家女', 0.7711703181266785)
('女朋友', 0.7703153491020203)
('表姐', 0.7699335813522339)
('男朋友', 0.7661849856376648)
('小妹', 0.7656798958778381)
('大姐', 0.76531308889